# Add Correction filing for All active existing companies

<b> Purpose: Add Corrections filing for all active existing BENs.</b>

This is a one time (python) script to be run at a given date/time.<br>
Set the configuration (client_id, client_secret, url(s)) for a scpecific environment.<br>
Get access token for authorization.


In [1]:
import requests
import os
from datetime import datetime

# token_url, client_id, client_secret, base_url - update based on environment
token_url = os.getenv('ACCOUNT_SVC_AUTH_URL')
client_id = os.getenv('ACCOUNT_SVC_CLIENT_ID')
client_secret = os.getenv('ACCOUNT_SVC_CLIENT_SECRET')
base_url = os.getenv('LEGAL_API_BASE_URL')

header = {
    "Content-Type": "application/x-www-form-urlencoded"
}

data = 'grant_type=client_credentials'

res = requests.post(token_url, data, auth=(client_id, client_secret), headers=header)

# Check the status code of the response
if res.status_code == 200:
    print(f"Access token returned successfully : {base_url}")
    token = res.json()["access_token"]
else:
    print(f"Failed to make POST request. Status code: {res.status_code}")
    print(res.text)  # Print the error message if the request fails



Access token returned successfully : https://legal-api-dev.apps.silver.devops.gov.bc.ca


Call API (POST) endpoint to createCorrection filing with details as Ben correction statement for businesses.

In [2]:
from urllib.parse import urljoin
from corrections_output import correction_businesses
from dateutil import parser

current_date_time = datetime.now()
current_date = datetime.now().date().isoformat()
formatted_current_date = datetime.now().date().strftime('%B %d, %Y')
correction_statement = ("BC benefit company statement contained in notice of articles as required under section " 
"51.992 of the Business Corporations Act corrected from “This company is a benefit company and, as such, has purposes "
"that include conducting its business in a responsible and sustainable manner and promoting one or more public "
"benefits” to “This company is a benefit company and, as such, is committed to conducting its business in a "
"responsible and sustainable manner and promoting one or more public benefits”")

headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + token
}

failed_identifiers = []

# loop through list of businesses to create filing
for correction_businesse in correction_businesses:
    identifier = correction_businesse[0]
    filind_id = correction_businesse[1]
    filing_date = correction_businesse[2]
    dt = parser.isoparse(filing_date)
    formatted_filing_date = dt.strftime("%B %d, %Y")
    
    correction_filing_data = {
        "filing": {
            "header": {
                "name": "correction",
                "date": current_date,
                "certifiedBy": "system",
                "correctionBenStatement": True,
                "waiveFees": True
            },
            "business": {
                "identifier": identifier,
                "legalType": "BEN"
            },
            "correction": {
                "details": "BEN Correction statement",
                "correctedFilingId": filind_id,
                "correctedFilingType": "incorporationApplication",
                "comment": f"""Correction for Incorporation Application filed on {formatted_filing_date} \n{correction_statement}"""
            }
        }
    }

    filing_url = urljoin(base_url, f"/api/v2/businesses/{identifier}/filings")
    rv = requests.post(filing_url, headers=headers, json=correction_filing_data)

    # Check the status code of the response
    if rv.status_code == 201:
        correction_filing_id = rv.json()["filing"]["header"]["filingId"]
        print(f"Correction created successfully for {identifier} and correction filing_id is {correction_filing_id}")
    else:
        failed_identifiers.append(identifier)
        print(f"Failed to make POST request. Status code: {rv.status_code}: {rv.text} for {identifier}")
print('Failed to file Corrections for:', failed_identifiers)  # Print the error message if the request fails  


Correction created successfully for BC0887768 and correction filing_id is 195706
Correction created successfully for BC0887642 and correction filing_id is 195707
Failed to file Corrections for: []
